In [1]:
import os
import numpy as np
import pandas as pd


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [5]:
train_dir = '../data/seg_train/seg_train'
test_dir = '../data/seg_test/seg_test'
pred_dir = '../data/seg_pred/seg_pred'

train_datagen = ImageDataGenerator(rescale=1./255  )
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
                        train_dir,
                        target_size=(150,150),
                        batch_size = 140,
                        class_mode='categorical'
                    )

Found 14034 images belonging to 6 classes.


In [ ]:
val_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size = (150,150),
                    batch_size = 30,    
                    class_mode = 'categorical'
                )

Found 3000 images belonging to 6 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break
    

data batch shape: (140, 150, 150, 3)
labels batch shape: (140, 6)


In [9]:
# build a basic Conv Net.

## instantiate basic Sequential model
model = keras.Sequential()

# add 1st Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(150,150,3)) )
model.add( Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same') )
model.add( MaxPooling2D( pool_size=(2,2) ) )

# add 2nd Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 128, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# add 3rd Conv layer (w/Max Pooling)
model.add( Conv2D(filters = 256, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 256, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# add 4th Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 512, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 512, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# flatten, hidden, and output layers
model.add( Flatten())
model.add( Dense(units=1024, activation='relu') )
model.add( Dense(units=512, activation='relu') )
model.add( Dense(units=6, activation='softmax') )

model.compile(
        loss = 'categorical_crossentropy',
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005),
        metrics = ['acc']
)

In [10]:
# take a look at the architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 73, 73, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                    

In [12]:
tboard = TensorBoard(log_dir='./logs', write_images=True, )
stopper = EarlyStopping(patience=10)
checkpts = ModelCheckpoint(filepath='./checkpoints/model-1/model-1', save_best_only=True, save_weights_only=True)



model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs = 25,
        validation_data = val_generator,
        validation_steps = 100,  # this parameter is very important because our val data is a generator: if not specified will repeat infinitely!
        callbacks = [tboard, stopper, checkpts]
)

Epoch 1/25
100/100 [==============================] - 19s 188ms/step - loss: 0.4577 - acc: 0.8318 - val_loss: 0.5467 - val_acc: 0.7937
Epoch 2/25
100/100 [==============================] - 19s 190ms/step - loss: 0.4184 - acc: 0.8494 - val_loss: 0.4284 - val_acc: 0.8507
Epoch 3/25
100/100 [==============================] - 18s 184ms/step - loss: 0.3683 - acc: 0.8655 - val_loss: 0.4392 - val_acc: 0.8430
Epoch 4/25
100/100 [==============================] - 18s 179ms/step - loss: 0.3202 - acc: 0.8869 - val_loss: 0.4816 - val_acc: 0.8313
Epoch 5/25
100/100 [==============================] - 18s 180ms/step - loss: 0.2971 - acc: 0.8951 - val_loss: 0.4572 - val_acc: 0.8470
Epoch 6/25
100/100 [==============================] - 18s 180ms/step - loss: 0.2356 - acc: 0.9164 - val_loss: 0.4769 - val_acc: 0.8503
Epoch 7/25
100/100 [==============================] - 18s 180ms/step - loss: 0.1908 - acc: 0.9303 - val_loss: 0.5061 - val_acc: 0.8580
Epoch 8/25
100/100 [==============================] - 1